In [1]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import sys
import time
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner
from executorch.exir import to_edge_transform_and_lower

# -----------------------------------
# 1. 모델 구조 정의 및 가중치 로드
# -----------------------------------
print("1. Loading the fine-tuned model structure and weights...")

# --- 설정값  ---
MODEL_REPO = "facebookresearch/dinov2"
MODEL_NAME = "dinov2_vits14"
# 클래스 수와 순서를 정확히 맞추기
NUM_CLASSES = 3 
CLASS_NAMES = ['downy','healthy', 'powdery']
DEVICE = torch.device("cpu")
print(f"--> Using device: {DEVICE}")
# --- 모델 구조 만들기  ---
try:
    # torch.hub를 이용해 DINOv2 모델 구조 로드
    model = torch.hub.load(MODEL_REPO, MODEL_NAME, pretrained=False) # pretrained=False로 설정
    
    # 마지막 분류기(머리)
    num_features = 384 # ViT-Small의 특징 벡터 크기
    model.head = nn.Linear(num_features, NUM_CLASSES)
    
    # --- 저장된 가중치 불러오기 ---
    model.load_state_dict(torch.load('/Cucumber-Disease-Prediction/AI/dinov2_hub_finetuned_model.pth', map_location=DEVICE))
    
    model = model.to(DEVICE)
    model.eval() # 평가 모드

    # 모델을 contiguous 포맷으로 강제 변환
    for param in model.parameters():
        param.data = param.data.contiguous()
    for buffer_name, buffer in model.named_buffers():
        model.register_buffer(buffer_name, buffer.contiguous())

    model = model.to(memory_format=torch.contiguous_format)

    print("--> Model loaded successfully.")


except Exception as e:
    print(f"\nERROR: Failed to load the model.")
    print(f"--> Ensure 'dinov2_hub_finetuned_model.pth' is in the same directory.")
    print(f"--> Original Error: {e}")
    sys.exit(1)

for name, param in model.named_parameters():
    if param.is_contiguous(memory_format=torch.channels_last):
        print(f"⚠️ Parameter {name} uses channels_last")

sample_inputs = (torch.randn(1, 3, 224, 224).contiguous(), )

edge_ir = torch.export.export(model, sample_inputs)


# export 후, executorch 변환 전에
for node in edge_ir.graph.nodes:
    if hasattr(node, 'meta') and 'memory_format' in node.meta:
        if node.meta['memory_format'] == torch.channels_last:
            print(f"⚠️ Node {node} uses channels_last")
            node.meta['memory_format'] = torch.contiguous_format

et_program = to_edge_transform_and_lower(
    edge_ir,
    partitioner=[XnnpackPartitioner()]
).to_executorch()

with open("vit_model.pte", "wb") as f:
    f.write(et_program.buffer)

/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/.venv/lib/python3.12/site-packages/executorch/exir/dialects/edge/_ops.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


1. Loading the fine-tuned model structure and weights...
--> Using device: cpu


Using cache found in /home/eden/.cache/torch/hub/facebookresearch_dinov2_main
/home/eden/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/eden/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/eden/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


--> Model loaded successfully.


In [5]:
import executorch
print(executorch.__version__)

AttributeError: module 'executorch' has no attribute '__version__'

In [4]:
from torch.utils.mobile_optimizer import optimize_for_mobile
# -----------------------------------
# 2. 모델을 TorchScript로 변환 (Tracing)
# -----------------------------------
print("\n2. Converting model to TorchScript...")

# DINOv2 (ViT-S)는 일반적으로 224x224 이미지를 입력으로 받습니다.
# (배치 크기 1, 채널 3, 높이 224, 너비 224)
# 만약 다른 크기로 fine-tuning 하셨다면 이 값을 변경해야 합니다.
dummy_input = torch.randn(1, 3, 224, 224).to(DEVICE)

try:
    # 1. Tracing: 모델에 더미 입력을 통과시켜 연산 그래프를 기록
    traced_script_module = torch.jit.trace(model, dummy_input)
    print("--> Tracing successful.")
    
    # 2. Optimize for Mobile: 모바일 환경에 맞게 모델 최적화
    optimized_lit_module = optimize_for_mobile(traced_script_module)
    print("--> Mobile optimization successful.")

    # 3. Save: 최적화된 .ptl 파일로 저장
    output_filename = "dinov2_mobile.ptl"
    optimized_lit_module.save(output_filename)
    
    print(f"\nSUCCESS! Model saved as '{output_filename}'")

except Exception as e:
    print(f"\nERROR: Failed to convert the model.")
    print(f"--> Original Error: {e}")
    sys.exit(1)


2. Converting model to TorchScript...
--> Tracing successful.
--> Mobile optimization successful.

SUCCESS! Model saved as 'dinov2_mobile.ptl'


In [16]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets # datasets 추가
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import sys

class_names = ['downy', 'healthy', 'powdery']

# ==============================================================
# 2. 모델 정의
# ==============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(pretrained=True)
# 출력층의 뉴런 개수를 데이터셋의 클래스 개수에 맞춰 자동으로 설정
model.classifier[1] = nn.Linear(model.last_channel, len(class_names))
model = model.eval().to(device)

model.load_state_dict(torch.load('/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/MobileNet/best_mobilenet_cucumber.pth', weights_only=False, map_location=device))

/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [34]:
# image.png를 입력해보는 코드

import PIL.Image as Image
import torchvision.transforms as transforms

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# 이미지 열기
img = Image.open("image.png").convert("RGB")
img_tensor = preprocess(img).unsqueeze(0).to(device)

random_input = torch.randn(1, 3, 224, 224)

# 추론
with torch.no_grad():
    output = model(random_input)

print(output)

tensor([[-3.8626,  5.7014, -2.9983]])


In [ ]:
from torch.utils.mobile_optimizer import optimize_for_mobile
# -----------------------------------
# 2. 모델을 TorchScript로 변환 (Tracing)
# -----------------------------------
print("\n2. Converting model to TorchScript...")

# DINOv2 (ViT-S)는 일반적으로 224x224 이미지를 입력으로 받습니다.
# (배치 크기 1, 채널 3, 높이 224, 너비 224)
# 만약 다른 크기로 fine-tuning 하셨다면 이 값을 변경해야 합니다.
dummy_input = torch.randn(1, 3, 224, 224).to(device)
model.eval()

OUTPUT_MODEL_NAME = "mobilenetCucumberMobile.ptl"
NO_OPT_MODEL_NAME = "mobilenetCucumberNoOpt.ptl"

# --- 4. TorchScript로 변환 및 모바일 최적화 ---
print(f"3. Converting model to TorchScript Lite (.ptl)...")

# ⚠️ 'torch.jit.trace' 대신 'torch.jit.script'를 사용합니다.
# 'script' 방식은 더미 입력(dummy_input)이 필요 없습니다.
try:
    # 1. Scripting: 모델의 코드를 직접 분석하여 변환
    print("--> Attempting torch.jit.script(model)...")
    scripted_module = torch.jit.script(model) # 🌟 이 부분이 변경됨
    print("--> Scripting successful.")

    scripted_module.save(NO_OPT_MODEL_NAME)
    
    # 2. Optimize for Mobile: 모바일 환경에 맞게 모델 최적화
    optimized_lit_module = optimize_for_mobile(scripted_module) # 🌟 입력 변수 변경
    print("--> Mobile optimization successful.")

    # 3. Save: 최적화된 .ptl 파일로 저장
    optimized_lit_module.save(OUTPUT_MODEL_NAME)
    
    print(f"\nSUCCESS! Model saved as '{OUTPUT_MODEL_NAME}'")
    print("--> This file is ready for your Kotlin (Android) app.")

except Exception as e:
    print(f"\nERROR: Failed to convert the model with 'torch.jit.script'.")
    print(f"--> Original Error: {e}")
    sys.exit(1)


2. Converting model to TorchScript...
3. Converting model to TorchScript Lite (.ptl)...
--> Attempting torch.jit.script(model)...
--> Scripting successful.
--> Mobile optimization successful.

SUCCESS! Model saved as 'mobilenetCucumberMobile.ptl'
--> This file is ready for your Kotlin (Android) app.


In [ ]:
import torch
import torchvision.models as models
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner
from executorch.exir import to_edge_transform_and_lower

class_names = ['downy', 'healthy', 'powdery']

sample_inputs = (torch.randn(1, 3, 224, 224), )


model = models.mobilenet_v2(pretrained=True)
# 출력층의 뉴런 개수를 데이터셋의 클래스 개수에 맞춰 자동으로 설정
model.classifier[1] = nn.Linear(model.last_channel, len(class_names))

model.load_state_dict(torch.load('/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/MobileNet/best_mobilenet_cucumber.pth', weights_only=False, map_location=device))
model.eval()

et_program = to_edge_transform_and_lower(
    torch.export.export(model, sample_inputs),
    partitioner=[XnnpackPartitioner()]
).to_executorch()

with open("vit_model.pte", "wb") as f:
    f.write(et_program.buffer)

InternalError: Failed with error: Tensor has a memory_format that is unsupported in ExecuTorch: torch.channels_last
Here is the node in the graph module:
graph():
    %p_cls_token : [num_users=1] = placeholder[target=p_cls_token]
    %p_pos_embed : [num_users=2] = placeholder[target=p_pos_embed]
    %p_blocks_0_norm1_weight : [num_users=1] = placeholder[target=p_blocks_0_norm1_weight]
    %p_blocks_0_norm1_bias : [num_users=1] = placeholder[target=p_blocks_0_norm1_bias]
    %p_blocks_0_ls1_gamma : [num_users=1] = placeholder[target=p_blocks_0_ls1_gamma]
    %p_blocks_0_norm2_weight : [num_users=1] = placeholder[target=p_blocks_0_norm2_weight]
    %p_blocks_0_norm2_bias : [num_users=1] = placeholder[target=p_blocks_0_norm2_bias]
    %p_blocks_0_ls2_gamma : [num_users=1] = placeholder[target=p_blocks_0_ls2_gamma]
    %p_blocks_1_norm1_weight : [num_users=1] = placeholder[target=p_blocks_1_norm1_weight]
    %p_blocks_1_norm1_bias : [num_users=1] = placeholder[target=p_blocks_1_norm1_bias]

In [2]:
import torch
from executorch.runtime import Runtime
from typing import List

runtime = Runtime.get()

input_tensor: torch.Tensor = torch.randn(1, 3, 224, 224)
program = runtime.load_program("vit_model.pte")
method = program.load_method("forward")
output: List[torch.Tensor] = method.execute([input_tensor])
print("Run successfully via executorch")

from torchvision.models.mobilenetv2 import MobileNet_V2_Weights
import torchvision.models as models

eager_reference_model = model.eval()
eager_reference_output = eager_reference_model(input_tensor)

print("Comparing against original PyTorch module")
print(torch.allclose(output[0], eager_reference_output, rtol=1e-3, atol=1e-5))

[program.cpp:134] InternalConsistency verification requested but not available
[tensor_util_portable.cpp:130] Check failed (all_contiguous || all_channels_last): 2 input tensors have different dim orders
[op_clone.cpp:41] Check failed (tensors_have_same_dim_order(self, out)): 
[method.cpp:1314] KernelCall failed at instruction 0:78 in operator aten::clone.out: 0x12
[method.cpp:1324] arg 0 with type id 1
[method.cpp:1324] arg 1 with type id 4
[method.cpp:1324] arg 2 with type id 1
[method.cpp:1324] arg 3 with type id 1


RuntimeError: method->execute() failed with error 0x12